# 1 - Package import

In [915]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from scipy import stats
from textwrap import wrap
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pyodbc 
import itertools

%matplotlib inline

# 2 - Data retrieval

In [1159]:
'''
country index mapping
'''
index_mapping={'AU':"AS51 Index",
        "AV":"SX5E Index",
        "BB":"SX5E Index",
        "CN":"SPX Index",
        "CT":"SPX Index",
        "DC":"SX5E Index",
        "FH":"SX5E Index",
        "FP":"SX5E Index",
        "GY":"SX5E Index",
        "HK":"HSI Index",
        "ID":"SX5E Index",
        "IM":"SX5E Index",
        "JP":"TPX Index",
        "JT":"TPX Index",
        "KS":"KOSPI Index",
        "LN":"SX5E Index",
        "NA":"SX5E Index",
        "NO":"SX5E Index",
        "PL":"SX5E Index",
        "SM":"SX5E Index",
        "SQ":"SX5E Index",
        "SS":"SX5E Index",
        "SW":"SX5E Index",
        "UN":"SPX Index",
        "US":"SPX Index",
        "VX":"SX5E Index"}

In [1160]:
def clean_sheet(dataframe):
    adj=dataframe.dropna(axis=1,how='all')
    adj=dataframe.set_index(adj.columns[0]) #use stock tickers as the index
    return adj

In [1483]:
US=clean_sheet(pd.read_excel(r'C:\Users\Eric.Li\Documents\Center Book\Raw US data.xlsx',\
                                       sheet_name='US'))

In [1162]:
def SQL_price(region,start,end):
    '''
    Grab the pricing data from SQL database
    region= US, Europe,Asia,or Canada
    start,end are in year
    key is the reference to search
    return the target price dataframe with timestamp on the column
    '''
    mylist=[]
    for year in range(start,end+1):
        sql="select * from {0}_price_{1}".format(region,year)
        data=pd.read_sql(sql,cnxn).set_index("Ticker")
        adj_data=data.loc[[x for x in data.index if len(x)>0]].replace('#N/A N/A','')
        mylist.append(adj_data)

    price=pd.concat(mylist,axis=1)
    price=price.apply(lambda x:pd.to_numeric(x),axis=1)
    
    sql_index="select * from {0}_price_index".format(region)
    data_index=pd.read_sql(sql_index,cnxn).set_index("Ticker").T
    data_index=data_index.replace('#N/A N/A','')
    price_index=data_index.apply(lambda x:pd.to_numeric(x),axis=1)
    
    price.columns=[SQL_price_date_transform(i) for i in price.columns]
    '''
    Need to sort the columns for index price, and then transform to date string
    '''
    price_index.columns=[SQL_price_date_transform(i,index=True) for i in price_index.columns]
    price_index=price_index.reindex(sorted(price_index.columns), axis=1)
    price_index.columns=[i.strftime("%d/%b/%Y") for i  in price_index.columns]
    
    abs_return=price.diff(1,axis=1)/price.shift(1,axis=1)
    abs_return_index=price_index.diff(1,axis=1)/price_index.shift(1,axis=1)
    return price,abs_return,price_index,abs_return_index

In [1163]:
def SQL_price_date_transform(SQL_date,index=False):
    '''
    Transform the SQL price style string into dateframe string style
    The SQL price follows US style which is MM DD YYYY
    '''
    if index==False:
        timestamp=pd.Timestamp(int(SQL_date[SQL_date.find(" ",3):]),
                            int(SQL_date[:SQL_date.find(" ")]),
                            int(SQL_date[SQL_date.find(" ",1):SQL_date.find(" ",3)]))
        return timestamp.strftime("%d/%b/%Y")
    else:
        timestamp=pd.Timestamp(int(SQL_date[-4:]),int(SQL_date[3:5]),int(SQL_date[:2]))
        return timestamp

In [1164]:
'''Connect to post result SQL server'''
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=dsa-ln-WS015\SQLEXPRESS;DATABASE=Post_result;Trusted_Connection=yes')   
cursor = cnxn.cursor()

In [1165]:
'''
Grab all the price data
'''
US_price,abs_return_US,US_index_price,abs_return_index_US=SQL_price('US',2006,2018)
Asia_price,abs_return_Asia,Asia_index_price,abs_return_index_Asia=SQL_price('Asia',2007,2018)
Europe_price,abs_return_Europe,Europe_index_price,abs_return_index_Europe=SQL_price('Europe',2006,2018)


In [1166]:
'''
Aggregate all the sub price data and replace it with dataframe
'''
index_return=abs_return_index_US.append(abs_return_index_Asia).append(abs_return_index_Europe)
index_return.columns=[datetime.strptime(x,"%d/%b/%Y") for x in index_return.columns]
index_return=index_return.drop_duplicates().T

In [1308]:
'''
Import the daily pnl data from FI Val
'''
FI_VAL_daily=clean_sheet(pd.read_excel(r'C:\Users\Eric.Li\Documents\Center Book\FI Val daily data.xlsx',sheet_name='Daily'))
FI_VAL_daily["Signal"]=list(zip(FI_VAL_daily["Ticker"],FI_VAL_daily["Start"]))
FI_price=clean_sheet(pd.read_excel(r'C:\Users\Eric.Li\Documents\Center Book\FI Val daily data.xlsx',sheet_name='Price'))
abs_return=FI_price.T.diff()/FI_price.T.shift(1)
abs_return=abs_return.T.drop_duplicates().T

In [1309]:
def unique_date(overlapping_df,column_name):
    '''
    transform the dataframe with overlapping date index to a dataframe with unique date
    '''
    my_df=pd.DataFrame()
    for i in sorted(set(overlapping_df.index.tolist())):
        data_row=overlapping_df[overlapping_df.index==i][[column_name,'Signal']].T
        data_row.columns=data_row.iloc[-1]
        data_row=data_row.iloc[:-1]
        data_row.index=[i]
        my_df=pd.concat([my_df,data_row],axis=0)      
    
    return my_df
        

In [1334]:
''' 
Set the capital size
'''
capital=10**(9)

In [1335]:
''' 
Update both pnl and size to unique date dataframe
'''

orig_FI_pnl=unique_date(FI_VAL_daily,'PNL')/capital
orig_FI_size=unique_date(FI_VAL_daily,'Exposure')/capital

In [1336]:
'''
Get rid of a position with holding period less than 5 days
'''
FI_count=orig_FI_pnl.copy()
FI_count[pd.notnull(FI_count)]=1

orig_FI_size=orig_FI_size.loc[:,FI_count.sum(axis=0)>5]
orig_FI_pnl=orig_FI_pnl.loc[:,FI_count.sum(axis=0)>5]

In [1337]:
'''
Add the country column and create the hedge  
Calculate both size and pnl
'''

FI_country=orig_FI_size.apply(lambda x:index_mapping[x.name[0][-2:]] if x.name[0][-2:] in index_mapping.keys() else None,axis=0)
adj_FI_size=orig_FI_size.T
adj_FI_size["country"]=FI_country
adj_FI_size=adj_FI_size[pd.notnull(adj_FI_size["country"])] #get rid of countries that are not in the mapping
alpha_FI_size=adj_FI_size.T.iloc[:-1]

# 3 - utility functions

In [1173]:
def trading_analytics_dollar(dollar_pnl_df,capital):
    '''
    dollar_pnl_df and capital
    '''
    
    ind_return=dollar_pnl_df.sum(axis=0)/capital
    signal_count=len(ind_return)
    account_curve=dollar_pnl_df.sum(axis=1).cumsum()/capital
    account_price=1+account_curve
    
    if signal_count==0:
        return None,None,None,None,None,None,None
    else:
        mean_return=ind_return.mean()
        hit_rate=len(ind_return[ind_return>0])/len(ind_return)*1.0
        payoff_ratio=ind_return[ind_return>0].mean()/ind_return[ind_return<0].mean()*-1.0
        
        account_price=account_curve+1
        ann_vol=np.std(account_price.diff()/account_price.shift(1))*(260**0.5)
        ann_ret=(account_price.iloc[-1]**(1/len(account_price)))**260-1
        ann_sharpe=ann_ret/ann_vol
        
        max_dd=-((1+account_curve)/(1+account_curve).cummax(axis=0)-1).expanding().min().min()
        
        #low_date=(np.maximum.accumulate(account_curve)-account_curve).idxmax()
        #high_date=account_curve[:low_date].idxmax()
        #max_dd=1-(1+account_curve[low_date])/(1+account_curve[high_date])
        
        return signal_count,hit_rate,payoff_ratio,ann_ret,ann_vol,ann_sharpe,max_dd

In [1174]:
def trading_analytics(daily_pnl_df):
    '''
    Key portfolio metrics from portfolio cache
    Feed into plot function
    '''
    
    ind_return=daily_pnl_df.sum(axis=0)
    signal_count=len(ind_return)
    account_curve=daily_pnl_df.cumsum().ffill().sum(axis=1)
    
    if signal_count==0:
        return None,None,None,None,None,None,None
    else:
        mean_return=ind_return.mean()
        hit_rate=len(ind_return[ind_return>0])/len(ind_return)*1.0
        payoff_ratio=ind_return[ind_return>0].mean()/ind_return[ind_return<0].mean()*-1.0
        
        account_price=account_curve+1
        ann_vol=np.std(account_price.diff()/account_price.shift(1))*(260**0.5)
        ann_ret=(account_price.iloc[-1]**(1/len(account_price)))**260-1
        ann_sharpe=ann_ret/ann_vol
        
        max_dd=-((1+account_curve)/(1+account_curve).cummax(axis=0)-1).expanding().min().min()
        
        #low_date=(np.maximum.accumulate(account_curve)-account_curve).idxmax()
        #high_date=account_curve[:low_date].idxmax()
        #max_dd=1-(1+account_curve[low_date])/(1+account_curve[high_date])
        
        return signal_count,hit_rate,payoff_ratio,ann_ret,ann_vol,ann_sharpe,max_dd

In [1354]:
def signal_account(return_df,index_return_df,orig_alpha_size,sizing,new_size):
    
    '''
    Build signal account from original size dataframe, and apply to different sizing scheme
    Sizing:
    - original
    - entry
    - average
    - equal
    - risk parity
    '''
    
    alpha_count_df=orig_alpha_size.copy()
    alpha_count_df[pd.notnull(alpha_count_df)]=1
        
    alpha_return_df=return_df[[x[0] for x in alpha_count_df.columns]].loc[alpha_count_df.index]
    alpha_return_df.columns=orig_alpha_size.columns
    alpha_signal_df=alpha_return_df*alpha_count_df
    
    if sizing=="original":
        alpha_size_df=orig_alpha_size
    else:
        if sizing=="entry":
            size_row=orig_alpha_size.apply(lambda x: x.dropna().iloc[0],axis=0)
        elif sizing=="average":
            size_row=orig_alpha_size.mean(axis=0)
        elif sizing=="equal":
            size_row=orig_alpha_size.apply(lambda x:new_size if x.mean()>0 else -new_size,axis=0)
        elif sizing=="risk parity":
            vol_row=orig_alpha_size.apply(lambda x:return_df[x.name[0]].loc[:x.name[1]][-30:].std(),axis=0)
            size_row=(vol_row.mean()/vol_row).fillna(1.0)*orig_alpha_size.apply(lambda x:new_size \
                                                                                if x.mean()>0 else -new_size,axis=0)
        
        alpha_size_df=(1+alpha_signal_df).cumprod()*size_row
    
    index_series=alpha_size_df.apply(lambda x:index_mapping[x.name[0][-2:]],axis=0)
    adj_alpha_size_df=alpha_size_df.T
    adj_alpha_size_df["Index"]=index_series
    
    alpha_size_df=adj_alpha_size_df.T.iloc[:-1]
    index_size_df=(adj_alpha_size_df.groupby("Index").apply(sum)*-1).T.iloc[:-1]
    index_signal_df=index_return_df[list(set(index_series.dropna()))].loc[alpha_signal_df.index]
    
    total_signal_df=pd.concat([alpha_signal_df,index_signal_df],axis=1)
    total_size_df=pd.concat([alpha_size_df,index_size_df],axis=1)
    
    total_daily_pnl=total_size_df.shift(1)*total_signal_df
    account_cuve=total_daily_pnl.cumsum().ffill().sum(axis=1)
    
    return total_daily_pnl,account_cuve,total_size_df
    

# 4 - Sizing

In [1259]:
def signal_account(return_df,index_return_df,size_row,orig_size_df):
    
    alpha_count_df=orig_size_df.copy()
    alpha_count_df[pd.notnull(alpha_count_df)]=1
    
    alpha_return_df=return_df[[x[0] for x in alpha_count_df.columns]].loc[alpha_count_df.index]
    alpha_return_df.columns=orig_size_df.columns
    alpha_signal_df=alpha_return_df*alpha_count_df
    
    alpha_size_df=(1+alpha_signal_df).cumprod()*(size_row*alpha_count_df)
    
    index_series=alpha_size_df.apply(lambda x:index_mapping[x.name[0][-2:]],axis=0)
    adj_alpha_size_df=alpha_size_df.T
    adj_alpha_size_df["Index"]=index_series
    
    alpha_size_df=adj_alpha_size_df.T.iloc[:-1]
    index_size_df=(adj_alpha_size_df.groupby("Index").apply(sum)*-1).T.iloc[:-1]
    index_signal_df=index_return_df[list(set(index_series.dropna()))].loc[alpha_signal_df.index]
    
    total_signal_df=pd.concat([alpha_signal_df,index_signal_df],axis=1)
    total_size_df=pd.concat([alpha_size_df,index_size_df],axis=1)
    
    total_daily_pnl=total_size_df.shift(1)*total_signal_df
    account_cuve=total_daily_pnl.cumsum().ffill().sum(axis=1)
    
    return total_daily_pnl,account_cuve,total_size_df
    

In [1355]:
daily_pnl,account_curve,size_df=signal_account(abs_return,index_return,alpha_FI_size,'equal',0.015)

C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5752: RuntimeWarning: '<' not supported between instances of 'str' and 'tuple', sort order is undefined for incomparable objects
  other.columns, how=join, level=level, return_indexers=True)


In [1359]:
long_FI_size=alpha_FI_size.loc[:,alpha_FI_size.apply(lambda x:x.mean(),axis=0)>0]

In [1360]:
short_FI_size=alpha_FI_size.loc[:,alpha_FI_size.apply(lambda x:x.mean(),axis=0)<0]

In [1363]:
trading_analytics(signal_account(abs_return,index_return,long_FI_size,'original',0.015)[0])

C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5752: RuntimeWarning: '<' not supported between instances of 'str' and 'tuple', sort order is undefined for incomparable objects
  other.columns, how=join, level=level, return_indexers=True)


(297,
 0.4444444444444444,
 2.021305039681683,
 0.033439286708573324,
 0.03331093025204848,
 1.0038532834584213,
 0.04064724174324996)

In [1364]:
trading_analytics(signal_account(abs_return,index_return,short_FI_size,'original',0.015)[0])

C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5752: RuntimeWarning: '<' not supported between instances of 'str' and 'tuple', sort order is undefined for incomparable objects
  other.columns, how=join, level=level, return_indexers=True)


(342,
 0.4152046783625731,
 1.5189555586142518,
 0.004654187723273084,
 0.021170467598921265,
 0.21984340693118357,
 0.038968761696676624)

In [1221]:
'''
Entry sizing 
'''
entry_FI_size_row=alpha_FI_size.apply(lambda x: x.dropna().iloc[0],axis=0)
daily_pnl,account_curve=signal_account(abs_return,index_return,entry_FI_size_row/1000000000,alpha_FI_size)

C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5752: RuntimeWarning: '<' not supported between instances of 'str' and 'tuple', sort order is undefined for incomparable objects
  other.columns, how=join, level=level, return_indexers=True)


# 5 - Attribution

In [1365]:
def quartile_name(target_series,large):
    '''
    Input input data: target_series to run quartile analysis, large ranks higher if true
    Output: pandas series that contains the quartile score
    '''
    quartile_series=target_series.copy()
    if large:
        quartile_series[target_series>target_series.quantile(0.8)]=1
        quartile_series[(target_series>target_series.quantile(0.6))&(target_series<=target_series.quantile(0.8))]=2
        quartile_series[(target_series>target_series.quantile(0.4))&(target_series<=target_series.quantile(0.6))]=3
        quartile_series[(target_series>target_series.quantile(0.2))&(target_series<=target_series.quantile(0.4))]=4
        quartile_series[target_series<=target_series.quantile(0.2)]=5
    else:
        quartile_series[target_series>target_series.quantile(0.8)]=5
        quartile_series[(target_series>target_series.quantile(0.6))&(target_series<=target_series.quantile(0.8))]=4
        quartile_series[(target_series>target_series.quantile(0.4))&(target_series<=target_series.quantile(0.6))]=3
        quartile_series[(target_series>target_series.quantile(0.2))&(target_series<=target_series.quantile(0.4))]=2
        quartile_series[target_series<=target_series.quantile(0.2)]=1
    return quartile_series

In [1501]:
def factor_rank_transform(fundamental_df):
    '''
    three factor version
    '''
    fundamental_df.loc[:,"mom"]=fundamental_df.apply(lambda x:x["RECENT PRICE"]/x["BEFORE PRICE"]-1 \
                                                     if type(x["BEFORE PRICE"])==float \
             and type(x["RECENT PRICE"])==float else None, axis=1)
    
    fundamental_df.loc[:,"value"]=fundamental_df.apply(lambda x:x["BOOK"]/x["Market Cap"] if type(x["BOOK"])==int \
             and type(x["Market Cap"])==int else None, axis=1)
    
    fundamental_df.loc[:,"size"]=fundamental_df.apply(lambda x: x["Market Cap"] if type(x["Market Cap"])==int else None, axis=1)
    
    fundamental_df=fundamental_df[np.isfinite(fundamental_df["mom"])]
    fundamental_df=fundamental_df[np.isfinite(fundamental_df["value"])]
    fundamental_df=fundamental_df[np.isfinite(fundamental_df["size"])]
    
    fundamental_df.loc[:,"size_rank"]=quartile_name(fundamental_df["size"],False)
    
    size_mom=pd.Series()
    for i in set(fundamental_df["size_rank"]):
        target=fundamental_df[fundamental_df["size_rank"]==i]
        target["size_mom"]=list(zip(target["size_rank"],quartile_name(target["mom"],True)))
        size_mom=size_mom.append(target["size_mom"])
    
    fundamental_df.loc[:,"size_mom"]=size_mom
    
    size_mom_value=[]
    for i in set(fundamental_df["size_mom"]):
        target=fundamental_df[fundamental_df["size_mom"]==i]
        target["size_mom_value"]=list(zip(list(zip(*target["size_mom"]))[0],list(zip(*target["size_mom"]))[1],\
                                          quartile_name(target["value"],True)))
        size_mom_value.append(target["size_mom_value"])
    
    
    fundamental_df.loc[:,"size_mom_value"]=pd.concat(size_mom_value,axis=0)
    
    return fundamental_df
    
    
    

In [1502]:
rank_dict=[]

for i in list(set(US["Date"]))[:3]:
    target_US=US[US["Date"]==i]
    rank_dict.append(factor_rank_transform(target_US)["size_mom_value"])

C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [1504]:
rank_dict[0]

Ticker
1624579D US Equity    (2, 2.0, 5.0)
1667208D US Equity    (1, 4.0, 4.0)
1694944D CN Equity    (5, 5.0, 3.0)
9191919D US Equity    (3, 3.0, 2.0)
9871234D US Equity    (4, 4.0, 1.0)
9999955D US Equity    (4, 4.0, 3.0)
9999966D US Equity    (4, 5.0, 4.0)
AA US Equity          (4, 1.0, 1.0)
AAAP US Equity        (3, 2.0, 5.0)
AAGIY US Equity       (5, 3.0, 3.0)
AAL US Equity         (5, 2.0, 4.0)
AAN US Equity         (3, 1.0, 1.0)
AAOI US Equity        (1, 1.0, 3.0)
AAON US Equity        (2, 3.0, 5.0)
AAP US Equity         (4, 5.0, 3.0)
AAPL US Equity        (5, 1.0, 4.0)
AAV CN Equity         (2, 4.0, 1.0)
AAWW US Equity        (2, 1.0, 1.0)
AAXN US Equity        (1, 4.0, 5.0)
ABAX US Equity        (1, 4.0, 4.0)
ABB US Equity         (5, 4.0, 4.0)
ABBV US Equity        (5, 3.0, 5.0)
ABC US Equity         (5, 5.0, 5.0)
ABCB US Equity        (2, 2.0, 3.0)
ABCO US Equity        (2, 2.0, 4.0)
ABEO US Equity        (1, 1.0, 4.0)
ABEV US Equity        (5, 4.0, 3.0)
ABG US Equity        

In [1422]:
US["size_rank"]=quartile_name(US["size"],False)

In [1433]:
size_mom=pd.Series()
for i in range(1,6):
    target=US[US["size_rank"]==i]
    target["size_mom"]=list(zip(quartile_name(target["mom"],True),target["size_rank"]))
    size_mom=size_mom.append(target["size_mom"])
    


C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
